In [68]:
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
import datetime as dt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder

### Read realtor data

In [18]:
realtor_2018_data = pd.read_csv(
    Path("realtor_data/2018_realtor_data.csv"))
realtor_2019_data = pd.read_csv(
    Path("realtor_data/2019_realtor_data.csv"))
display(realtor_2018_data)

,sold_date,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size
0,2018-01-02,for_sale,450000.0,3.0,2.0,0.09,"15 William St, North Arlington, NJ, 07031",15 William St,North Arlington,New Jersey,7031.0,NaN
1,2018-01-02,for_sale,234900.0,3.0,2.0,0.87,"185 Gilsum St, Keene, NH, 03431",185 Gilsum St,Keene,New Hampshire,3431.0,1752.0
2,2018-01-02,for_sale,234900.0,3.0,2.0,0.87,"185 Gilsum St, Keene, NH, 03431",185 Gilsum St,Keene,New Hampshire,3431.0,1752.0
3,2018-01-02,for_sale,1245000.0,1.0,1.0,NaN,"57 Reade St Apt 5D, New York, NY, 10007",57 Reade St Apt 5D,New York,New York,10007.0,785.0
4,2018-01-02,for_sale,234900.0,3.0,2.0,0.87,"185 Gilsum St, Keene, NH, 03431",185 Gilsum St,Keene,New Hampshire,3431.0,1752.0
...,...,...,...,...,...,...,...,...,...,...,...,...
25191,2018-12-31,for_sale,389000.0,1.0,1.0,NaN,"62 Park Ter W Apt A26, New York, NY, 10034",62 Park Ter W Apt A26,New York,New York,10034.0,800.0
25192,2018-12-31,for_sale,3995000.0,2.0,3.0,NaN,"200 E 62nd St Apt 14E, New York, NY, 10065",200 E 62nd St Apt 14E,New York,New York,10065.0,1936.0
25193,2018-12-31,for_sale,399000.0,NaN,NaN,1.15,"1 Mayer Ln, Pomona, NY, 10970",1 Mayer Ln,Pomona,New York,10970.0,NaN
25194,2018-12-31,for_sale,3475000.0,2.0,2.0,NaN,"200 Mercer St Apt 4E, New York City, NY, 10012",200 Mercer St Apt 4E,New York City,New York,10012.0,2500.0


In [19]:
csv_files = glob.glob('realtor_data/*')
csv_files

['realtor_data\\2018_realtor_data.csv', 'realtor_data\\2019_realtor_data.csv']

### Merge realtor data

In [20]:
realtor_data = pd.DataFrame()

for file in csv_files:
            df_temp = pd.read_csv(file)
            realtor_data = realtor_data.append(df_temp)
            
realtor_data.set_index("zip_code", drop=True, inplace=True)

display(realtor_data)

C:\Users\Maria Angels Bonany\AppData\Local\Temp\ipykernel_1604\2874440782.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  realtor_data = realtor_data.append(df_temp)
C:\Users\Maria Angels Bonany\AppData\Local\Temp\ipykernel_1604\2874440782.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  realtor_data = realtor_data.append(df_temp)


,sold_date,status,price,bed,bath,acre_lot,full_address,street,city,state,house_size
zip_code,,,,,,,,,,,
7031.0,2018-01-02,for_sale,450000.0,3.0,2.0,0.09,"15 William St, North Arlington, NJ, 07031",15 William St,North Arlington,New Jersey,NaN
3431.0,2018-01-02,for_sale,234900.0,3.0,2.0,0.87,"185 Gilsum St, Keene, NH, 03431",185 Gilsum St,Keene,New Hampshire,1752.0
3431.0,2018-01-02,for_sale,234900.0,3.0,2.0,0.87,"185 Gilsum St, Keene, NH, 03431",185 Gilsum St,Keene,New Hampshire,1752.0
10007.0,2018-01-02,for_sale,1245000.0,1.0,1.0,NaN,"57 Reade St Apt 5D, New York, NY, 10007",57 Reade St Apt 5D,New York,New York,785.0
3431.0,2018-01-02,for_sale,234900.0,3.0,2.0,0.87,"185 Gilsum St, Keene, NH, 03431",185 Gilsum St,Keene,New Hampshire,1752.0
...,...,...,...,...,...,...,...,...,...,...,...
6450.0,2019-12-31,for_sale,259900.0,3.0,2.0,0.18,"20 Russell Rd, Meriden, CT, 06450",20 Russell Rd,Meriden,Connecticut,1312.0
6112.0,2019-12-31,for_sale,299999.0,NaN,NaN,0.18,"439 Edgewood St, Hartford, CT, 06112",439 Edgewood St,Hartford,Connecticut,3922.0
2127.0,2019-12-31,for_sale,1598000.0,3.0,2.0,NaN,"57 L St Apt 10, Boston, MA, 02127",57 L St Apt 10,Boston,Massachusetts,1574.0


### Merge IRS data

In [21]:
csv_files = glob.glob('IRS_Income_tax_data/*.csv')
csv_files

['IRS_Income_tax_data\\IRS_2018_data_sliced.csv',
 'IRS_Income_tax_data\\IRS_2019_data_sliced.csv']

In [22]:
IRS_sliced_data = pd.DataFrame()

for file in csv_files:
            df_temp = pd.read_csv(file)
            IRS_sliced_data = IRS_sliced_data.append(df_temp, ignore_index=True)
            
display(IRS_sliced_data)

C:\Users\Maria Angels Bonany\AppData\Local\Temp\ipykernel_1604\4150780073.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  IRS_sliced_data = IRS_sliced_data.append(df_temp, ignore_index=True)
C:\Users\Maria Angels Bonany\AppData\Local\Temp\ipykernel_1604\4150780073.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  IRS_sliced_data = IRS_sliced_data.append(df_temp, ignore_index=True)


,STATE,zipcode,A18425
0,AL,0,2905.0
1,AL,0,27454.0
2,AL,0,56235.0
3,AL,0,60309.0
4,AL,0,292798.0
...,...,...,...
332089,WY,99999,0.0
332090,WY,99999,0.0
332091,WY,99999,0.0
332092,WY,99999,122.0


In [23]:
# Set the Index to zip-code
IRS_sliced_data.rename(columns = {'zipcode':'zip_code'}, inplace=True)
IRS_sliced_data.set_index('zip_code', drop=True, inplace=True)

display(IRS_sliced_data)

,STATE,A18425
zip_code,,
0,AL,2905.0
0,AL,27454.0
0,AL,56235.0
0,AL,60309.0
0,AL,292798.0
...,...,...
99999,WY,0.0
99999,WY,0.0
99999,WY,0.0


### Merge realtor and IRS data on zip_code (left join)

In [55]:
df1 = realtor_data.merge(IRS_sliced_data, how = 'left', on = "zip_code")
df1.drop(columns=["state"], inplace = True)

## The final DataFrame we are going to work with!!!

In [70]:
display(df1)

,sold_date,status,price,bed,bath,acre_lot,full_address,street,city,house_size,STATE,A18425
zip_code,,,,,,,,,,,,
7031.0,2018-01-02,for_sale,450000.0,3.0,2.0,0.09,"15 William St, North Arlington, NJ, 07031",15 William St,North Arlington,NaN,NJ,0.0
7031.0,2018-01-02,for_sale,450000.0,3.0,2.0,0.09,"15 William St, North Arlington, NJ, 07031",15 William St,North Arlington,NaN,NJ,134.0
7031.0,2018-01-02,for_sale,450000.0,3.0,2.0,0.09,"15 William St, North Arlington, NJ, 07031",15 William St,North Arlington,NaN,NJ,366.0
7031.0,2018-01-02,for_sale,450000.0,3.0,2.0,0.09,"15 William St, North Arlington, NJ, 07031",15 William St,North Arlington,NaN,NJ,663.0
7031.0,2018-01-02,for_sale,450000.0,3.0,2.0,0.09,"15 William St, North Arlington, NJ, 07031",15 William St,North Arlington,NaN,NJ,1989.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7086.0,2019-12-31,for_sale,919000.0,4.0,2.0,NaN,"5 Shippen St, Weehawken, NJ, 07086",5 Shippen St,Weehawken,NaN,NJ,230.0
7086.0,2019-12-31,for_sale,919000.0,4.0,2.0,NaN,"5 Shippen St, Weehawken, NJ, 07086",5 Shippen St,Weehawken,NaN,NJ,221.0
7086.0,2019-12-31,for_sale,919000.0,4.0,2.0,NaN,"5 Shippen St, Weehawken, NJ, 07086",5 Shippen St,Weehawken,NaN,NJ,438.0


# Encode the dataset’s categorical variables using OneHotEncoder

In [71]:
# Create a list of categorical variables 
categorical_variables = list(df1.dtypes[df1.dtypes == "object"].index)

In [72]:
display(categorical_variables)

['sold_date', 'status', 'full_address', 'street', 'city', 'STATE']

In [73]:
enc = OneHotEncoder(sparse=False)

In [80]:
encoded_data = enc.fit_transform(df1[categorical_variables])

MemoryError: Unable to allocate 64.0 GiB for an array with shape (593111, 14481) and data type float64

In [81]:
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names_out(categorical_variables)
)

NameError: name 'encoded_data' is not defined

In [82]:
display(encoded_df)

NameError: name 'encoded_df' is not defined

# Apply Logistic Regression

# Split training into testing sets (Create X, or features DataFrame, and create y, or target DataFrame)

In [58]:
features = df1[['STATE', 'A18425']]

target = df1['price']

In [59]:
training_features, testing_features, training_targets, testing_targets = train_test_split(features, target)

In [60]:
training_features

,STATE,A18425
zip_code,,
10027.0,NY,90.0
7305.0,NJ,542.0
11217.0,NY,111053.0
3285.0,NH,0.0
7438.0,NJ,325.0
...,...,...
10019.0,NY,19049.0
19018.0,PA,1580.0
11207.0,NY,9832.0


# Create and use a classifier that can predict whether the house sold price price will be higher or lower

In [61]:
logistic_regression_model = LogisticRegression()

# Fit: Train the Model by supplying it with some training it

In [62]:
logistic_regression_model.fit(training_features, training_targets)

ValueError: could not convert string to float: 'NY'

# Generate predictions from the model we just fit

In [63]:
predictions = logistic_regression_model.predict(training_features)

ValueError: could not convert string to float: 'NY'

In [64]:
results_df = pd.DataFrame({"Prediction": predictions, "Actual": training_targets})
results_df

NameError: name 'predictions' is not defined

# Apply the fitted model to the test dataset

In [65]:
testing_predictions = logistic_regression_model.predict(testing_features)

ValueError: could not convert string to float: 'MA'

In [ ]:
results_df = pd.DataFrame({
    "Testing Data Predictions": testing_predictions,
    "Testing Data Actual Targets": testing_targets})
results_df

# Compare each predicted value to its actual value

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(testing_targets, testing_predictions)

# We evaluated the model predictions. If high accuracy (closer to 1) it may ean that there is overfitting which may mean that the model won't perform well on new data it was not trained on

# We can categorize the predictions on higher house prices or lower house prices according to a confusion matrix.